### this script has been taken from the checkFiltering and DivideTrialsIntoReadingAndWriting script, and so all the filtering is based on the same

In [3]:
%matplotlib qt
#%matplotlib inline
import os
import csv
import numpy as np
import re
import datetime
import fnmatch
from pathlib import Path
from itertools import groupby
import copy
import itertools
#import distance
import nltk
#nltk.download('stopwords')
import pandas as pd
import matplotlib.pyplot as plt
import pickle

from nltk.stem import SnowballStemmer
from nltk.stem import snowball

from itertools import *
from operator import *

from openpyxl import load_workbook

In [4]:
# exceptional removal of particular extra sentences not typed by the user 
dict_phraseStim = {
    #'2019-02-05-14-10-39_2ndPart_2' : [1, 2, 3, 4, 5, 6, 9, 10],
    #'2019-01-14-14-58-30' : [0], # ys, session_trial ()
    '2019-01-16-16-36-17_1stPart_2' : [-1], # af_session1
    '2019-01-16-17-00-12_2ndPart_2': [1], # af_session1
    '2019-01-17-15-27-20_1stPart_2' : [4], # Af session2
    '2019-01-17-16-03-27_2ndPart_2' : [0, 1, 2], # Af session2
    '2019-02-06-11-25-41_1' : [7],               # aq_session1    
    '2019-02-08-11-33-53_1stPart_1' : [1],  # aq session3_1_part1
    '2019-02-08-12-11-34_2ndPart_1' : [0, 1, 2, 3],  # aq session3_1_part2
    '2019-01-31-09-37-5_2ndPart_2' : range(1,5), # bh1, session 4 , all sentences except the first one deleted
    '2019-01-31-09-22-49_1stPart_2' : [4],  # bh1_session4_2_part1
    '2019-02-21-16-09-44_1stPart_1' : [1], # bh2_session1
    '2019-02-21-16-22-22_2ndPart_1' : [2, 3, 4],# bh2_session1
    '2019-02-28-17-03-53_1stPart_2' : [2],       # bh2_session3
    '2019-02-28-17-24-2_2ndPart_2' : [0, 2],     # bh2_session3
    '2019-02-14-13-28-20_1stPart_2' : [2], # cw_session3_2_part1
    '2019-02-14-13-57-41_2ndPart_2' : [0, 2, 3], # cw_session3_2_part2
    '2019-02-21-15-01-4_1stPart_1' : [0],        # le_session3
    '2019-02-21-15-25-56_2ndPart_1' : [1],        # le_session3
    '2019-02-18-10-28-35_2' : [0],               # ls2_session4 # picture not described
    '2019-02-05-14-00-27_1stPart_2' : [3],        # mh_session1
    '2019-02-05-14-10-39_2ndPart_2' : [0, 1, 3],   # mh_session1
    '2019-02-08-10-51-3_1stPart_1' : [4],        # mn_session1
    '2019-02-08-11-05-7_2ndPart_1' : [0, 2, 3, 4], # mn_session1
    '2019-02-19-10-34-7_1stPart_1' : [3],          # mn_session3
    '2019-02-19-10-56-43_2ndPart_1' : [1, 2, 3, 4], # mn_session3
    '2019-01-16-15-18-0_1' : [4],            # no_session1
    '2019-02-19-17-10-45_1' : [3],                  # ph_session5
    '2019-01-29-13-25-4_1' : [3],        # ph_session2
    '2019-03-07-16-44-5_2' : [1],                   # rh_session1
    '2019-03-14-13-56-56_2' : [2],                  # rh_session3
    '2019-01-14-15-07-21_1' : [4], # ys_session1
    '2019-01-16-15-18-50_1stPart_1' : [3, 4], # ys_session2
    '2019-01-16-15-42-51_2ndPart_1' : [2], # ys_session2
    '2019-01-30-11-22-25_1' : [3, 5, 7],          # ys_session4
    '2019-01-30-11-22-25_1' : [4, 6, 7] # ys, session 4
}

# exceptional removal of sentences/words typed by the user, but then deleted everything to have a blank scratchpad

dict_phraseUser = {
    "2019-02-06-15-44-15_1" : [2, 3, 6], 
    "2019-02-06-16-19-9_2" : [1, 3, 6, 7],
    "2019-02-12-11-21-21_2" : [0],
    "2019-02-14-14-28-49_1" : [0, 2, 3], # ac_session3_1
    "2019-02-14-14-45-49_2" : [0, 5, 6], # ac_session3_2
    '2019-01-29-14-19-26_1' : [0, 3, 4], # bh1_session2_1
    '2019-01-29-14-40-36_2' : [0, 1, 2], # bh1_session2_2
    '2019-01-30-14-29-29_2' : [4],       # bh1_session3_2
    '2019-01-31-09-12-2_1' : [3],         # bh1_session4_1
    '2019-01-31-09-22-49_1stPart_2' : [4], # bh1_session4_2_part1
    '2019-03-05-09-15-11_1' : [1],         # bh2_session5_1
    '2019-03-05-09-15-11_2' : [1],        # bh2_session5_2
    '2019-02-21-15-55-56_2' : [2],       # ch_session5_2
    '2019-01-30-15-19-36_2' : [1],       # jm_session2_1
    '2019-01-30-15-04-30_1' : [0],         # jm_session2_2
    '2019-01-16-15-18-50_1stPart_1' : [1],  # ys_session2
    '2019-01-16-15-42-51_2ndPart_1' : [0], # ys_session2
    '2019-01-30-11-22-25_1' : [2, 4],       # ys_session4
    '2019-01-30-11-57-3_2' : [0] ,          # ys_session4
    '2019-01-31-13-13-2_1' : [4],           # ys_session5
    '2019-01-30-10-20-32_1' : [0, 1, 2, 3, 4, 5], # no_session4
    '2019-01-30-10-46-38_2' : [0],          # 
    '2019-02-28-17-03-53_1stPart_2' : [2],   # bh2_session3
    '2019-03-12-09-30-5_1' : [0],            # kj_session3
    '2019-02-13-15-20-38_1' : [0, 1, 2, 3, 6], # ls1_session3
    '2019-02-18-10-25-52_1' : [1],              # ls2_session4
    '2019-02-18-10-46-26_2' : [0],            # ls2_session4
    '2019-01-29-13-25-4_1' : [0, 1, 7],        # ph_session2
    '2019-01-29-13-43-50_2' : [0],              # ph_session2
    '2019-03-07-16-17-30_1' : [0],              # rh_session1
    '2019-03-07-16-44-5_2' : [0, 1],         # rh_session1
    '2019-03-14-13-56-56_2' : [0, 1, 3]         # rh_session3
}

# key selection can have extra selections of NextPhrase at the end
dict_keySelectionOfNextPhrase = {
    "2019-02-11-11-18-30_1" : [12, 13], # ac_session1
    "2019-01-16-17-00-12_2ndPart_2" : [12], # af_session1
    "2019-01-17-15-27-20_1stPart_2" : [12], # af_session2
    "2019-02-06-16-19-9_2" : [12], # af_session3
    "2019-02-12-11-07-43_1" : [12], # af_session4
    "2019-02-27-15-08-32_1" : [12], # af_session5
    "2019-01-28-14-30-44_1" : [12], # bh1_session1
    "2019-02-21-16-22-22_2ndPart_1" : [12], # bh2_session1
    "2019-02-18-14-02-56_2" : [12], # le_session1
    "2019-02-19-10-03-14_1" : [12], # le_session2
    "2019-02-08-11-05-7_2ndPart_1" : [12], # mn_session1
    "2019-02-08-11-12-51_2" : [12, 13], # mn_session1
    "2019-02-15-11-38-22_1" : [12, 13], # mn_session2
    "2019-02-15-11-54-25_2" : [12], # mn_session2
    "2019-01-16-15-18-0_1" : [12], # no_session1
    "2019-01-28-13-31-51_1" : [12], # ph_session1
    "2019-01-28-13-49-14_2" : [12], # ph_session1
    "2019-01-14-15-07-21_1" : [12], # ys_session1
    "2019-01-17-15-05-1_1" : [12], # ys_session3
    "2019-01-30-11-22-25_1" : [12], # ys_session4
    "2019-01-31-13-32-2_2" : [12], # ys_session5
}


# key selection when participants skips some sentences
dict_keySelectionNotCompleted = {
    "2019-01-16-16-36-17_1stPart_2" : [0, 1, 3, 5, 7], # af_session1 ---- last sentence is not finished
    "2019-01-16-17-00-12_2ndPart_2" : [0, 1, 3, 4, 5, 7, 9, 11], # af_session1
    "2019-01-17-15-27-20_1stPart_2" : [0, 1, 3, 5, 7, 9, 11], # af_session2 
    "2019-01-17-16-03-27_2ndPart_2" : [0, 1, 2, 3, 4, 5, 6, 7, 9, 11], # af_session2
    "2019-02-08-11-33-53_1stPart_1" : [0, 1, 3, 4, 5, 7, 9, 11], # aq_session3
    "2019-02-08-12-11-34_2ndPart_1" : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11], # aq_session3
    "2019-01-28-14-30-44_1" : [0, 1, 3, 5], # bh1_session1
    "2019-01-31-09-22-49_1stPart_2": [0, 1, 3, 5, 7, 9, 10, 11], # bh1_session4
    "2019-01-31-09-37-5_2ndPart_2" : [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11], # bh1_session4
    "2019-02-21-16-09-44_1stPart_1" : [0, 1, 3, 4, 5, 7, 9, 11], # bh2_session1
    "2019-02-21-16-22-22_2ndPart_1" : [0, 1, 3, 5, 6, 7, 8, 9, 10, 11], # bh2_session1
    "2019-02-28-17-03-53_1stPart_2" : [0, 1, 3, 5, 6, 7, 9, 11], # bh2_session3
    "2019-02-28-17-24-2_2ndPart_2" : [0, 1, 2, 3, 5], # bh2_session3     ----
    "2019-02-14-13-28-20_1stPart_2" : [0, 1, 3, 5, 6, 7, 9, 11], # cw_session3
    "2019-02-14-13-57-41_2ndPart_2" : [0, 1, 2, 3, 5, 6, 7, 8, 9, 11], # cw_session3
    "2019-02-21-15-01-4_1stPart_1" : [0, 1, 2, 3, 5, 7, 9, 11], # le_session3
    "2019-02-21-15-25-56_2ndPart_1" : [0, 1, 3], # le_session3       ----
    "2019-02-05-14-00-27_1stPart_2" : [0, 1, 3, 5, 7, 8], # mh_session1
    "2019-02-05-14-10-39_2ndPart_2" : [0, 1, 2, 3, 4, 5, 7, 8, 9, 11], # mh_session1
    "2019-02-08-10-51-3_1stPart_1" : [0, 1, 3, 5, 7, 9, 10, 11], # mn_session1
    "2019-02-08-11-05-7_2ndPart_1" : [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11], # mn_session1
    "2019-02-19-10-34-7_1stPart_1" : [0, 1, 3, 5, 7, 8, 9, 11], # mn_session3
    "2019-02-19-10-56-43_2ndPart_1" : [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11], # mn_session3
    "2019-01-29-13-25-4_1" : [0, 1, 3, 5, 7], # ph_session2  -- sessions where there are less score questions 
    # and more sentences typed
    "2019-01-16-15-18-50_1stPart_1" : [0, 1, 3, 5, 7, 8, 9, 10], # ys_session2
    "2019-01-17-15-05-1_1" : [0, 1, 3, 5],  # ys_session3  -- sessions where there are less score questions 
    # and more sentences typed
    "2019-02-06-11-25-41_1" : [0, 1, 3, 5, 11], # aq_session1 -- sessions where there are less score questions 
    # and more sentences typed
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 2, 5], # ys_session2 -- different for reading and writing, this one is for
    # writing
    '2019-01-30-11-22-25_1' : [0, 1, 3, 5, 7, 9, 11]   # ys_session4 -- sessions where there are less score questions 
    # and more sentences typed
   
}

# dictionary for phrase removal just like in the dict_phraseStim, but since not all participants require that, some that 
# do, are added to this new dictionary here
dict_keySelection_phraseStim = {
    '2019-01-17-15-27-20_1stPart_2' : [4], # Af session2
    '2019-01-16-15-18-0_1' : [4],        # no_session1
    '2019-02-19-17-10-45_1' : [3],                  # ph_session5
    '2019-03-07-16-44-5_2' : [1],        # rh_session1
    '2019-03-14-13-56-56_2' : [2],              # rh_session3
    '2019-01-14-15-07-21_1' : [4]         # ys_session1
}


# in the beginning experiments, not everyone started with 800 initial dwell time

dict_dwellTimeOrig_not800 = {
    "2019-01-16-15-51-13_2" : 600, # no_session1
    "2019-01-16-15-18-0_1" : 600, # no_session1
    "2019-01-16-15-43-8_1" : 100, # af_session1
    "2019-01-16-16-36-17_1stPart_2" : 100, # af_session1
    "2019-01-16-17-00-12_2ndPart_2" : 100, # af_session1
    "2019-01-17-15-03-40_1" : 100, # af_session2
    "2019-01-17-15-27-20_1stPart_2" : 0, # af_session2
    "2019-01-17-16-03-27_2ndPart_2" : 100, # af_session2
    "2019-01-14-15-07-21_1" : 500, # ys_session1
    "2019-01-14-15-25-55_2" : 300, # ys_session1
    "2019-01-16-15-18-50_1stpart_1" : 200, # ys_session2
    "2019-01-16-15-42-51_2ndPart_1" : 100, # ys_session2
    "2019-01-16-15-59-55_2" : 100, # ys_session2
    "2019-01-17-15-05-1_1" : 100, # ys_session3
    "2019-01-17-15-31-12_2" : 100 # ys_session3
}


# list of all things that should be present when computing effective time
list_keysToBeCounted = ['Comma', 'BackOne', 'BackMany', 'SpaceBar']

# some sessions do not have gaze data
dict_noGazeData = {
    '2019-01-16-17-00-12_2ndPart_2' : 'no gaze data', # af_session2
    '2019-01-17-15-31-12_2' : 'no gaze data', #ys_session2
    '2019-01-30-11-57-3_2' : 'no gaze data' # ys_session4
}



In [5]:
dict_keySelection_ReadingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 3, 5], # ys_session2 
}

dict_keySelection_WritingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 2, 5], # ys_session2   
}

# normally, reading part of trial ends when people look at the keyboardwithphrases. For some trials, this is not done,
# as the reading is done, and the trial is accidentally skipped, and written in the next trial. Here, the trial number 
# given will have the reading time ending as sleep, and not keyboard with phrases. 
dict_keyboardNotChange_ReadingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : 0, # ys_session2 
}

dict_keySelection_firstSleepNotCounted = {
    "2019-01-28-14-50-41_2" : (0, 2), # bh1_session1 -- 3rd sleep activation to be counted
    "2019-02-19-10-56-43_2ndPart_1" : 2  # mn_session3 -- 3rd sleep activation is to be counted
}

In [6]:
TimeDwellOrig = 800
TimeFixation = 300

In [7]:
def FixUserKeys(UserKeys_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    Column_beforeDecimal = [item[2] for item in UserKeys_Old]
    Column_afterDecimal = [item[3] if len(item)>3 else '00' for item in UserKeys_Old]
    
    UserKeys_ProgressPercent = [float(Column_beforeDecimal[i]+'.'+ Column_afterDecimal[i]) for i in 
                                range(0, len(Column_beforeDecimal))]
    UserKeys_Times = [item[0] for item in UserKeys_Old]
    UserKeys_Keys = [item[1] for item in UserKeys_Old]
    
    UserKeys_New = [[UserKeys_Times[ind], UserKeys_Keys[ind], UserKeys_ProgressPercent[ind]] for ind in 
                    range(0, len(UserKeys_ProgressPercent))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    
    return UserKeys_New
        

In [8]:
def FixKeysSelected(KeysSelected_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    KeysSelected_New = list()
    
    # loop to combine phrases divided by commas
    KeysSelectedInd = -1 
    while KeysSelectedInd < len(KeysSelected_Old)-1:
        KeysSelectedInd = KeysSelectedInd + 1
        
        if KeysSelected_Old[KeysSelectedInd][1].count(',') > 0:
            
            keys_split = KeysSelected_Old[KeysSelectedInd][1].split("\r\n")
            del keys_split[0]
            del keys_split[-1]
            
            keys_split = [key.split(',') for key in keys_split]
            
            KeysSelected_New.extend(keys_split)
        else:
            KeysSelected_New.append(KeysSelected_Old[KeysSelectedInd])
        
    
    return KeysSelected_New

In [9]:
def ComputeDwellTime(userKeys, full_path):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    TimeDwellOrig = 800
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    if session_folder_name in dic_dwellTimeOrig_not800:
        TimeDwellOrig = dic_dwellTimeOrig_not800[session_folder_name]
    
    #print(TimeDwellOrig)
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        #print(key[1])
        if key[1] == 'IncreaseDwellTime':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseDwellTime':
            #print(key[2])
            if float(key[2]) == 1:
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [10]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [11]:
def findAndRemoveTrials(session_name, dictionary_saved, trials, replacingList):
    # function to check the session_name in the dictionary_saved and remove those trials from the dictionary_trial
    
    if session_name in dictionary_saved:
        index_list = dictionary_saved[session_name]
    else:
        index_list = replacingList
    
    
    

    if index_list:
        if type(trials) == list:
            for index in sorted(index_list, reverse=True):
                del trials[index]
                
        else:
            for index in sorted(index_list, reverse=True):
                del trials['start'][index]
                del trials['end'][index]
            
    return trials    

In [12]:
# function to convert list of date and time into datetime format list
def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [13]:
def OptiKeyTypingTime(UserKeys):
    
    TimeTyping = dict()
    
    time1, t1, t2 = UserKeys[0][0].partition('+')
    startTime = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    time2, t1, t2 = UserKeys[-1][0].partition('+')
    endTime = datetime.datetime.strptime(re.sub('[:.T]','-',time2[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    TimeTyping['startTime'] = startTime
    TimeTyping['endTime'] = endTime
    
    return TimeTyping

In [14]:
def FindExptStartEndTimes(KeysSelected, timeTyping, full_path):
    # function to find start and end of tasks in experiments
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    timeTrialDict = dict()
    timeTrialDict = {'start': [],
                    'end':[]}
    
    nTrial = -1
    
    
    
    for keys in KeysSelected:
        
            
        
        if keys[1] == 'NextPhrase':
            nTrial = nTrial + 1
            time1, t1, t2 = keys[0].partition('+')
            endTimeTrial = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            
            if nTrial != 0:
                # print('end: ', endTimeTrial)
                #print('')
                timeTrialDict['end'].append(endTimeTrial)
            
            
            startTimeTrial = endTimeTrial + datetime.timedelta(seconds=5)
            
            #print('start: ', startTimeTrial)
            timeTrialDict['start'].append(startTimeTrial)
        
    del timeTrialDict['start'][-1]
    
    
    # remove the extra selections of NewPhrase at the end of some sessions
    replacingList = []
    timeTrialDict = findAndRemoveTrials(session_folder_name, dict_keySelectionOfNextPhrase, timeTrialDict, replacingList)
    
    timeTrialDict_copy = copy.deepcopy(timeTrialDict)
    
    # separate the reading and writing trials for some participants who read in the actual trial, but write in the next
    # trial
    if session_folder_name in dict_keySelection_ReadingTrials:
        # check the reading and writing separate dictionaries
        print('reading and writing sessions are separate')
        
        #print(len(timeTrialDict['start']))
        # writing trials - 
        timeTrialDict_writing = findAndRemoveTrials(session_folder_name, dict_keySelection_WritingTrials, timeTrialDict, replacingList)
        #print(len(timeTrialDict_copy['start']))
        
        # reading trials
        timeTrialDict_reading = findAndRemoveTrials(session_folder_name, dict_keySelection_ReadingTrials, timeTrialDict_copy, replacingList)
    else:
        # some participants skip some sentences, and then it affects the scoreQuestions too. Remove the skipped sentences or 
        # remove the score questions 
        # for these participants, reading and writing trials are the same
        
        #print('same reading and writing trials')
        scoreQuestions = [0, 1, 3, 5, 7, 9, 11]
        timeTrialDict = findAndRemoveTrials(session_folder_name, dict_keySelectionNotCompleted, timeTrialDict, scoreQuestions)
        
        # most of the skipped sentences are removed, but for those that are not removed
        timeTrialDict_writing = findAndRemoveTrials(session_folder_name, dict_keySelection_phraseStim, timeTrialDict, replacingList)
        
        timeTrialDict_reading = timeTrialDict_writing 
        
    
          
    return timeTrialDict_reading, timeTrialDict_writing 

In [15]:
def FindReadingPartsOfTrial_inKeysSelected(EventTrials_reading, KeysSelected_new, full_path):
    # find the reading end in the pupil size data
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    KeysSelected_timeStr = [key[0] for key in KeysSelected_new]
    KeysSelected_time = timeConversion(KeysSelected_timeStr)
    
    KeysSelected_keys = [key[1] for key in KeysSelected_new]
    
    EventReading = dict()    
    EventReading['start'] = list()
    EventReading['end'] = list()
    
    EventReading_index = dict()    
    EventReading_index['start'] = list()
    EventReading_index['end'] = list()
    
    for ind, startTrialTime_afterCoolDown in enumerate(EventTrials_reading['start']):
        
        startTrialTime, startTrialInd = nearestTimePoint(KeysSelected_time, startTrialTime_afterCoolDown)
        
        EventReading['start'].append(startTrialTime)
        EventReading_index['start'].append(startTrialInd)
        
        endTrialTime = EventTrials_reading['end'][ind]
        endTrialInd = KeysSelected_time.index(endTrialTime)
        
        keysSelected_trial = KeysSelected_keys[startTrialInd:endTrialInd]
        
        # compute the end of reading time -- which is taken when the keyboard with phrases key is pressed
        if session_folder_name not in dict_keyboardNotChange_ReadingTrials:
            
            endReading_keyInd = startTrialInd + keysSelected_trial.index('KeyboardWithPhrases') - 1
            endReading_keyTime = KeysSelected_time[endReading_keyInd]
            
        else:
            if ind == dict_keyboardNotChange_ReadingTrials[session_folder_name]:
                endReading_keyInd = startTrialInd + keysSelected_trial.index('Sleep') - 1
                endReading_keyTime = KeysSelected_time[endReading_keyInd]
                
            else:
                endReading_keyInd = startTrialInd + keysSelected_trial.index('KeyboardWithPhrases') - 1
                endReading_keyTime = KeysSelected_time[endReading_keyInd]
                
                
        EventReading['end'].append(KeysSelected_time[endReading_keyInd+1])
        EventReading_index['end'].append(endReading_keyInd+1)
        
        #print(ind)
        #print('reading: ', EventReading['start'][ind], EventReading['end'][ind])
        
    
    
    return EventReading, EventReading_index

In [16]:
def FindWritingPartsOfTrial_inKeysSelected(EventTrials_writing, KeysSelected_new, EventReading):
    
    KeysSelected_timeStr = [key[0] for key in KeysSelected_new]
    KeysSelected_time = timeConversion(KeysSelected_timeStr)
    
    KeysSelected_keys = [key[1] for key in KeysSelected_new]
    
    EventWriting = dict()    
    EventWriting['start'] = list()
    EventWriting['end'] = list()
    
    EventWriting_index = dict()    
    EventWriting_index['start'] = list()
    EventWriting_index['end'] = list()
    
    for ind, startTrialTime_afterCoolDown in enumerate(EventTrials_writing['start']):
        
        startTrialTime, startTrialInd = nearestTimePoint(KeysSelected_time, startTrialTime_afterCoolDown)
        
        endTrialTime = EventTrials_writing['end'][ind]
        endTimeReading = EventReading['end'][ind]
        
        
        # for some participants, reading and writing trials are different. So their writing times will not be the end of 
        # the reading time.
        # Regardless, the writing time should start later than when the reading time ends.
        # We choose the starting time for writing as the one that is later than the start time from writing trials
        # and end time from reading trials
        
        if startTrialTime > endTimeReading:
            EventWriting['start'].append(startTrialTime)
            EventWriting_index['start'].append(startTrialInd)
        else:
            EventWriting['start'].append(endTimeReading)
            EventWriting_index['start'].append(KeysSelected_time.index(endTimeReading))
        
        EventWriting['end'].append(endTrialTime)
        EventWriting_index['end'].append(KeysSelected_time.index(endTrialTime))
        
        #print(ind)
        #print('writing: ', EventWriting['start'][ind], EventWriting['end'][ind])
        
    
    
    return EventWriting, EventWriting_index     

In [17]:
def gazeConvert2ColumnsTo1(GazeLog, columnIndwValidity_list, indValidity):
    # function to convert pupilsizes from 2 columns for every pupil due to comma use instead of decimal, 
    # to proper pupil sizes
    
    #columnInd_list = [joinColumn1_1, joinColumn1_2, joinColumn2_1, joinColumn2_2]
    
    # number of columns in the final dictionary
    nColumns = int(len(columnIndwValidity_list)/2)
    
    # dictionary of columns that are to be joined later
    columns_beforeDecimal = dict()
    columns_afterDecimal = dict()
    
    # dictionary of joined columns
    columnsFinal = dict()
    
    # dictionary to find and equalize missing values in every column
    missingVal_column = dict()
    missingVal = list()
    
    # find correct index of validity column to be used, to find the actual columns relative to that
    columnsValidity_inUse = list()
    
    for ind, row in enumerate(GazeLog):
        #print(ind)
        #print(sorted(list(np.where(np.array(row) == 'Valid')[0])+list(np.where(np.array(row)=='Invalid')[0]))[indValidity])

        columnsValidity = (sorted(list(np.where(np.array(row) == 'Valid')[0])+list(np.where(np.array(row)=='Invalid')[0]))[indValidity])
        columnsValidity_inUse.append(int(columnsValidity))
    
    columnsValidity_inUse = np.array(columnsValidity_inUse)
    
    columnInd_list = [[columnsValidity_inUse+i] for i in columnIndwValidity_list]
    
    
    for ind in range(0, nColumns):
        
        dict_name = 'column' + str(ind+1)
        columnsFinal[dict_name] = list()
        columns_afterDecimal[dict_name] = list()
                
        #for indItem, item4 in enumerate(GazeLog):
        #    if 'Invalid' not in item4:
        #        if columnInd_list[2*ind+1][0][indItem] < len(item4):
        #            columns_afterDecimal[dict_name].append(item4[columnInd_list[2*ind+1][0][indItem]])
        #        else:
        #            columns_afterDecimal[dict_name].append('0')
        #    else:
        #        columns_afterDecimal[dict_name].append('nan')
        
                
        columns_beforeDecimal[dict_name] = [item4[columnInd_list[2*ind][0][indItem]] if 'Invalid' not in item4 else 'nan' for indItem, item4 in enumerate(GazeLog)]
        columns_afterDecimal[dict_name] = [item4[columnInd_list[2*ind+1][0][indItem]] if 'Invalid' not in item4 and columnInd_list[2*ind+1][0][indItem] < len(item4) else 'nan' for indItem, item4 in enumerate(GazeLog)]

        
        for i in range(0, len(columns_beforeDecimal[dict_name])):
            if 'Valid' not in columns_beforeDecimal[dict_name][i] and 'Valid' not in columns_afterDecimal[dict_name][i]:
                if 'nan' not in columns_beforeDecimal[dict_name][i] and 'nan' not in columns_afterDecimal[dict_name][i]:
                    if float(columns_afterDecimal[dict_name][i]) > 0: 
                        columnsFinal[dict_name].append(float(columns_beforeDecimal[dict_name][i]+'.'+columns_afterDecimal[dict_name][i]))
                    else:
                        columnsFinal[dict_name].append(np.nan)
                else:
                    columnsFinal[dict_name].append(np.nan)
            else:
                # Rarely, the pupil size is a whole number
                columnsFinal[dict_name].append(np.nan) # we will ignore the row, since there is no way of automatically knowing which - 
                # right or left eye has whole number pupil size
    
        missingVal_column[dict_name] = np.argwhere(np.isnan(columnsFinal[dict_name]))
        missingVal_column[dict_name] = list(itertools.chain.from_iterable(missingVal_column[dict_name])) # flatten the list
        
        missingVal.extend(missingVal_column[dict_name])
        
        
    
    missingVal = sorted(set(missingVal))
    
    # if one of the columns are nan, the other one is converted too
    for column in range(0, nColumns):
        dict_name = 'column' + str(column+1)
        for ind in missingVal:
            if ind < len(columnsFinal[dict_name]):
                columnsFinal[dict_name][ind] = np.nan
                
    
    return columnsFinal

In [18]:
def hampel(dvec, radius=5, nsigma=3, rem_nomed=False):

    # replace outliers with median values (hampel filter)
    
    mvec = pd.Series(dvec).rolling(radius*2+1, center=True, min_periods=radius).median()
    svec = 1.4862 * np.abs(dvec-mvec).rolling(radius*2+1, center=True, min_periods=radius).median()
    plonk = np.abs(dvec-mvec) > nsigma*svec
    dvec = np.array(dvec)
    dvec[plonk.tolist()] = mvec[plonk.tolist()]

    # remove "bad data" where we cannot calculate a median value due to already missing values
    if (rem_nomed):
        dvec[np.isnan(mvec)] = np.nan
    return dvec

In [19]:
def filterBlinks_wBlinkData(pupilDataL, pupilDataR, timeInDatetime):
    # filter any blinks and nan values lasting around 250ms (on average)
    # first the single nan occurances are replaced with mean of the values on either sides, 
    # as they are assumed to be from hardware problems
    # for the rest of the blinks, 250ms before and after the nan values are interpolated with a linear function
    # returns a dataframe with pupil size, and timestamp
    # http://faculty.washington.edu/chudler/facts.html
   
    
    # create a dataframe from the pupilsize and time
    pupilData_df = pd.DataFrame(list(zip(timeInDatetime, pupilDataL, pupilDataR)), columns=['timeStamp', 'pupilLeft', 'pupilRight'])
    
    # blink is every nan value in the range of 100-400ms 
    # 250 ms (22 samples) before and after the blink will also be removed
    extraBlinkSamples = 18
    
    
    #pupilData_woSingleMissingData = pupilData.copy()
    #timeList_woSingleMissingData = timeInDatetime.copy()
    #timeInS_woSingleMissingData = timeInS_Trial[-1]
    
    # in case of single missing data, that are due to hardware error, replace with the mean of the pupil size before and
    # after the nan value
    # missing values will be the same for left and right pupil
    missingVal_Single = np.argwhere(np.isnan(pupilDataL))
    missingVal_Single = list(itertools.chain.from_iterable(missingVal_Single)) # flatten the list 
    
    
    
    # if no blinks present, return the data
    if len(missingVal_Single) == 0:
        interpolatedNan_bool = np.array([False]*len(pupilData_df['pupilLeft']))
        return pupilData_df, interpolatedNan_bool
    
    # find the index and values to replace for single nan values
    pupilData_tuples_replaceSingleNan_left = [(val, np.mean([pupilDataL[val-1], pupilDataL[val+1]])) for i, val in enumerate(missingVal_Single) if (val != 0 and val != (len(pupilDataL)-1)) if not np.isnan(pupilDataL[val-1]) and not np.isnan(pupilDataL[val+1])]
    pupilData_tuples_replaceSingleNan_right = [(val, np.mean([pupilDataR[val-1], pupilDataR[val+1]])) for i, val in enumerate(missingVal_Single) if (val != 0 and val != (len(pupilDataR)-1)) if not np.isnan(pupilDataR[val-1]) and not np.isnan(pupilDataR[val+1])]
    
    interpolatedNan_bool = np.array([True if ind in dict(pupilData_tuples_replaceSingleNan_left) else False for ind, val in enumerate(pupilDataL)])
    missingData_single = interpolatedNan_bool
    
    # replace the single nan values with the mean of the pupil size on either sides
    indList = -1
    for ind, val in pupilData_tuples_replaceSingleNan_left:
        indList = indList + 1
        pupilData_df.iloc[ind, pupilData_df.columns.get_loc('pupilLeft')] = val
        pupilData_df.iloc[ind, pupilData_df.columns.get_loc('pupilRight')] = pupilData_tuples_replaceSingleNan_right[indList][1]
        
    
    # again, find the nan values in the pupil size
    # the list missingVal_SingleDifference contains the index of the first blink, followed by the difference in the index
    # to the next nan value
    
    
    
    # find the nan values again from pupilData['pupilLeft']
    missingVal_Rest_trueFalse = pupilData_df['pupilLeft'].isnull()
    missingVal_Rest = [i for i, x in enumerate(missingVal_Rest_trueFalse) if x]
    
    # if no blinks left, return the current pupilData
    if len(missingVal_Rest) == 0:
        return pupilData_df, interpolatedNan_bool

    
    # in the blinks left, find when the blinks start by finding a difference in the consecutive values of the indices
    missingVal_RestDifference = [t - s for s, t in zip(missingVal_Rest, missingVal_Rest[1:])]
    missingVal_RestDifference.insert(0, missingVal_Rest[0])
    
    blinkStart_tupleList = [(ind, sum(missingVal_RestDifference[0:ind+1])) for ind, val in enumerate(missingVal_RestDifference) if val != 1]
    
    blinkStart_tupleList_wLength = list()
    
    # create a list of tuples of blink start index and the length of the blink
    ind = -1
    blinkLengthSum = 0
    for blink_ind, blinkStartInd in blinkStart_tupleList:
        ind = ind + 1
        if ind != len(blinkStart_tupleList) - 1:
            
            blinkLength = blinkStart_tupleList[ind+1][0]-blink_ind
            blinkLengthSum = blinkLengthSum + blinkLength
            
            blinkStart_tupleList_wLength.append(tuple((blinkStartInd, blinkLength)))
        else:
            # for the last blink -- all blink lengths summed and subtracted from the length of the list
            # missingVal_RestDifference 
            blinkLength = len(missingVal_RestDifference)-blinkLengthSum
            blinkStart_tupleList_wLength.append(tuple((blinkStartInd, blinkLength)))
     
    
    
    
    # create a vector with True if a blink was at the position
    samplingFrequency = 90
    blinkLengthMax = np.ceil(0.5*samplingFrequency)
    blinkLengthMin = np.ceil(0.075*samplingFrequency)
    blinkStart_tupleList_wLength_allMissingData = [(blinkStart, blinkLength) for blinkStart, blinkLength in blinkStart_tupleList_wLength]    
    blinkStart_tupleList_wLength_maxBlinkLength = [(blinkStart, blinkLength) for blinkStart, blinkLength in blinkStart_tupleList_wLength if blinkLength<blinkLengthMax and blinkLength>blinkLengthMin]
    
    missingData_blinks_bool = np.array([False]*len(pupilData_df['pupilLeft']))
    for blinkStart, blinkLength in blinkStart_tupleList_wLength_maxBlinkLength:
        blinkIndices = np.arange(blinkStart, blinkStart + blinkLength).astype(int)
        missingData_blinks_bool[blinkIndices] = True
        
    
    
    missingData_blinks_boolAll = np.array([False]*len(pupilData_df['pupilLeft']))
    for blinkStart, blinkLength in blinkStart_tupleList_wLength:
        blinkIndices = np.arange(blinkStart, blinkStart + blinkLength).astype(int)
        missingData_blinks_boolAll[blinkIndices] = True
        
    missingDataOverall_bool = missingData_single + missingData_blinks_boolAll
    
    # add to vector with start and end of tuple
    #beforeAfterNan = [False]*len(pupilData_df['pupilLeft'])
    #for blinkStart, blinkLength in blinkStart_tupleList_wLength:
    #    beforeAfterNan[blinkStart] = True
    #    beforeAfterNan[blinkStart+blinkLength] = True
    #    #print('start and end points: ', pupilData_df['timeStamp'][blinkStart], pupilData_df['timeStamp'][blinkStart + blinkLength])
    
    
    # create lists with start and end values for the blinks, based on blinkStart_tupleList_wLength, regardless of the blink length
    blink_missingData_startList = [blinkStartInd - extraBlinkSamples if (blinkStartInd - extraBlinkSamples) > 0 else 0 for blinkStartInd, blinkLength in blinkStart_tupleList_wLength]
    blink_missingData_endList = [blinkStartInd + blinkLength + extraBlinkSamples if (blinkStartInd + blinkLength + extraBlinkSamples) < (len(pupilData_df['pupilLeft'])-1) else (len(pupilData_df['pupilLeft'])-1) for blinkStartInd, blinkLength in blinkStart_tupleList_wLength]
    # create a list of tuples from the start and end points of the blink
    blink_missingData_startEndTuple = [(blinkStart, blink_missingData_endList[ind]) for ind, blinkStart in enumerate(blink_missingData_startList)] 
    
    
    # check if blinks need to be combined - blinksCombine is a list of list of 2 elements, the index of the blinks that should be combined
    blinksCombine = [[ind, ind+1] for ind, blink in enumerate(blink_missingData_startEndTuple[0:-1]) if blink[1] > blink_missingData_startEndTuple[ind+1][0]]
        
    if blinksCombine:
        # combine blinks that need to be combined - if multiple consecutive blinks need to be removed: eg - [1, 2], [2, 3] 
        # are included in the blinksCombine, the combined version should be [1, 3] 
        blinksCombineFinal = list()
        ind = -1
        while ind < len(blinksCombine)-2:
            
            ind = ind + 1
            blinkCombining = blinksCombine[ind]
            blinksCombineFinal.append(blinkCombining)
            while ind < len(blinksCombine)-2 and blinkCombining[1] == blinksCombine[ind+1][0]:
                # change the ending of the last added blink of blinksCombineFinal
                blinksCombineFinal[-1][1] = blinksCombine[ind+1][1]
                ind = ind + 1
            
            
        if len(blinksCombine) == 1:
            blinksCombineFinal = blinksCombine.copy()
            
        
        if blinksCombine[-1][1] != blinksCombineFinal[-1][1]:
            if blinksCombine[-1][0] == blinksCombineFinal[-1][1]:
                blinksCombineFinal[-1][1] = blinksCombine[-1][1]
            else:
                blinksCombineFinal.append(blinksCombine[-1])
        
        
        
        
    #    for w, z in groupby(sorted(list(blinksCombine)), lambda x, y=itertools.count(): next(y)-x):
    #        group = list(z)
    #        blinksCombineFinal.append(tuple((group[0], group[-1])))
        
        for x in sorted(blinksCombineFinal, reverse=True):
            new_start = blink_missingData_startEndTuple[x[0]][0] 
            new_end = blink_missingData_startEndTuple[x[1]][1] 
            
            x_start = x[0]
            x_end = x[1]
            
            # delete also the blinkStart_tupleList_wLength, since it is going to be used to compute other metrics
            for blinkRemove in range(x[1], x[0]-1, -1):
                del blink_missingData_startEndTuple[blinkRemove]
            
            blink_missingData_startEndTuple.insert(x[0], tuple((new_start, new_end)))
    
    
    #blinkAndNonBlinkDurationList = [length/90 for start, length in blinkStart_tupleList_wLength]
    #timeInS_Trial_filter = timeInS_Trial[-1] - sum(blinkAndNonBlinkDurationList) 
    
    
    # remove blinks from data
    for blinkStart, blinkEnd in blink_missingData_startEndTuple:
        pupilData_df.loc[blinkStart:blinkEnd,'pupilLeft'] = np.nan
        pupilData_df.loc[blinkStart:blinkEnd,'pupilRight'] = np.nan
        replaceTrueList = range(blinkStart, blinkEnd+1, 1)
        interpolatedNan_bool[replaceTrueList] = True
    
    
    
    pupilData_df['pupilLeft'] = pupilData_df['pupilLeft'].astype(float).interpolate('linear', limit_direction = 'both')
    pupilData_df['pupilRight'] = pupilData_df['pupilRight'].astype(float).interpolate('linear', limit_direction = 'both')
    
    if pupilData_df.isnull().any().any():
        print('nan values in filtered data')
        #for i,val in enumerate(pupilData_filter[0:5000]):
        #    print(i, val, pupilData_woSingleMissingData[i])
        
    
    return pupilData_df, interpolatedNan_bool, missingData_blinks_bool, missingDataOverall_bool, blinkStart_tupleList_wLength_maxBlinkLength, blinkStart_tupleList_wLength_allMissingData

In [20]:
def FilterPupilSize_wBlinkData(GazeLog, TimeTyping, subjectAndSessionName):
    # function that uses the list of start and end trial times to find the pupil sizes for those trials and plots them
    
    # first create a list of times in gaze log
    timeStrGazeLog = [item3[0] for item3 in GazeLog]
    # convert the list of strings to datetime formats
    timeGazeLog = timeConversion(timeStrGazeLog)
    
    # internal time, to depict seconds
    timeInternalGazeLog = [float(item3[1]) for item3 in GazeLog]
    
    # extract pupil sizes in decimals from the strange 2 columns for every pupil
    pupil_indWrtValidityL = [1, 2]
    pupil_validityL = 4
    pupilLogL_raw = gazeConvert2ColumnsTo1(GazeLog, pupil_indWrtValidityL, pupil_validityL)
    
    pupil_indWrtValidityR = [1, 2]
    pupil_validityR = 5
    pupilLogR_raw = gazeConvert2ColumnsTo1(GazeLog, pupil_indWrtValidityR, pupil_validityR)
    
    
    # reduce the data to start and end of typing time
    timeTyping_start, timeTyping_startInd = nearestTimePoint(timeGazeLog, TimeTyping['startTime'])
    timeTyping_end, timeTyping_endInd = nearestTimePoint(timeGazeLog, TimeTyping['endTime'])
    
    
    pupilLogL_wDefinedTime = pupilLogL_raw['column1'][timeTyping_startInd:timeTyping_endInd+1]
    pupilLogR_wDefinedTime = pupilLogR_raw['column1'][timeTyping_startInd:timeTyping_endInd+1]
    
    timeGazeLog_wDefinedTime = timeGazeLog[timeTyping_startInd:timeTyping_endInd+1]
    
    
    
    timeInS_GazeLog_wDefinedTime = timeInternalGazeLog[timeTyping_startInd:timeTyping_endInd+1]
    timeInS_Difference = [(t - s)/1000000 for s, t in zip(timeInS_GazeLog_wDefinedTime, timeInS_GazeLog_wDefinedTime[1:])]
    timeInS_Difference.insert(0, 0)
    
    
    #timeInS = [sum(timeInS_Difference[:i]) for i, v in enumerate(timeInS_Difference)]
    
    pupilData_df, interpolated_items, missingData_fromBlinks, missingData, missingDataBlink_startNlength, \
    missingDataAll_startNlength = filterBlinks_wBlinkData(pupilLogL_wDefinedTime, pupilLogR_wDefinedTime, timeGazeLog_wDefinedTime)
    
    
    
    
    #timeGazeLog_plot = np.arange(0, timeInS[-1], 1/90)
    
    #plotPupilSize_checkFilter(pupilData_df, pupilLogL_wDefinedTime, blinkStartAndEnd, 'blink removal', subjectAndSessionName)
    
    
    
    
    pupilData_df_hampel = dict()
    
    pupilData_df_hampel = pupilData_df.copy()
    pupilData_df_hampel['pupilLeft'] = hampel(pupilData_df['pupilLeft'], 25, 3, False)
    pupilData_df_hampel['pupilRight'] = hampel(pupilData_df['pupilRight'], 25, 3, False)
        
    
        
    return pupilData_df_hampel, interpolated_items, missingData_fromBlinks, missingData, missingDataBlink_startNlength, missingDataAll_startNlength

In [21]:
def EventPartsFromPupilData(EventTimeInKeys, PupilSize_df, full_path):
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    EventTime = dict()    
    EventTime['start'] = list()
    EventTime['end'] = list()
    
    EventIndex = dict()    
    EventIndex['start'] = list()
    EventIndex['end'] = list()
    
    EventBaseline_startKeyTime = list()
    
    for ind, eventStartInKeys in enumerate(EventTimeInKeys['start']):
        
        eventStartTime, eventStartInd = nearestTimePoint(PupilSize_df['timeStamp'].tolist(), eventStartInKeys)
        eventEndTime, eventEndInd = nearestTimePoint(PupilSize_df['timeStamp'].tolist(), EventTimeInKeys['end'][ind])
        
        # reading start is the same as trial start
        EventTime['start'].append(eventStartTime)
        EventIndex['start'].append(eventStartInd)
        
        EventTime['end'].append(eventEndTime)
        EventIndex['end'].append(eventEndInd)
        
        
    return EventTime, EventIndex    

In [22]:
def CombineReadingWriting(EventReading, EventWriting, EventReading_index, EventWriting_index):
    
    EventTrial = copy.deepcopy(EventReading)
    EventTrialEnd = [endTime for endTime in EventWriting['end']]
    EventTrial['end'] = EventTrialEnd
    
    EventTrial_index = copy.deepcopy(EventReading_index)
    EventTrialEnd_index = [endTime for endTime in EventWriting_index['end']]
    EventTrial_index['end'] = EventTrialEnd_index
    
    
    return EventTrial, EventTrial_index

In [51]:
def GetBlinkPropertiesForEvents_onSections(EventTrial_index, PupilData_df, MissingData_startNlength):
    
    samplingFrequency = 90
    nSections = 4
    
    
    blinkFrequencyListSection = list()
    blinkDurationAverageListSection = list()
    blinkDurationTotalListSection = list()
    blinkCountListSection = list()
    interBlinkDurationListSection = list()
        
    
    # for every trial event, find the blinks during that event
    for ind, eventStart in enumerate(EventTrial_index['start']):
        eventEnd = EventTrial_index['end'][ind]
        
        blinkFrequencyList = list()
        blinkDurationAverageList = list()
        blinkDurationTotalList = list()
        blinkCountList = list()
        interBlinkDurationList = list()
        
        nSamples = eventEnd - eventStart + 1
        
        # blink properties for each of the 4 sections of the trial
        for nSection in range(nSections):
            eventStartSection = eventStart + nSection*int(nSamples/4)
            eventEndSection = eventStart + (nSection+1)*int(nSamples/4)
            
            blinkInd_missingData = np.array([indBlink for indBlink in range(0, len(MissingData_startNlength)) if \
                MissingData_startNlength[indBlink][0] > eventStartSection and MissingData_startNlength[indBlink][0] < \
                                         eventEndSection])
            blinkDurations_trial = np.array([MissingData_startNlength[indBlink][1] for indBlink in blinkInd_missingData])
            interBlinkDuration_trial = [MissingData_startNlength[indBlink][0] - MissingData_startNlength[indBlink-1][0] - \
                                  MissingData_startNlength[indBlink-1][1] for indBlink in blinkInd_missingData[1:]]
        
        
        
            trialTime = ((PupilData_df['timeStamp'][eventEndSection] - PupilData_df['timeStamp']\
                          [eventStartSection]).total_seconds())/60
            
            
            blinkCountList.append(len(blinkInd_missingData))
            blinkFrequencyList.append(len(blinkInd_missingData)/trialTime)
            
            blinkDurationTotalList.append(sum(blinkDurations_trial)/samplingFrequency)
            if len(interBlinkDuration_trial) > 0:
                interBlinkDurationList.append(np.mean(interBlinkDuration_trial)/samplingFrequency)
            else:
                interBlinkDurationList.append(0.0)
                
            if len(blinkDurations_trial) > 0:
                blinkDurationAverageList.append(np.mean(blinkDurations_trial)/samplingFrequency)
            else:
                blinkDurationAverageList.append(0.0)
            
            MissingData_startNlength_array = np.array(MissingData_startNlength)
            
        
        
        blinkCountListSection.append(blinkCountList)
        blinkFrequencyListSection.append(blinkFrequencyList)
        blinkDurationAverageListSection.append(blinkDurationAverageList)
        blinkDurationTotalListSection.append(blinkDurationTotalList)
        interBlinkDurationListSection.append(interBlinkDurationList)
        
        
        
        #print(MissingData_startNlength_array[blinkInd_missingData])
        #print(len(blinkInd_missingData)/trialTime)
        #print(sum(blinkDurations_trial)/samplingFrequency)
        #print(np.mean(blinkDurations_trial)/samplingFrequency)
        
    Blink_df = pd.DataFrame(list(zip(blinkCountListSection, blinkFrequencyListSection, blinkDurationAverageListSection, \
                                     blinkDurationTotalListSection,interBlinkDurationListSection)), columns=[\
                                    'blinkCount', 'blinkFrequency', 'blinkDurationAverage', 'blinkDurationTotal', \
                                    'interBlinkDuration'])
    Blink_df = Blink_df.fillna(0)
    
    
    return Blink_df

In [52]:
class DataForEveryTrial:
    subjectID = ''
    blockNumber = ''
    sessionNumber = ''
    variable = ''
    dataForTrial = ''
    resultPathName = ''
   
    
    def printInfo(self):
        dataFrame = pd.DataFrame(list(zip([self.subjectID]*len(self.dataForTrial), [self.blockNumber]*len(self.dataForTrial), [self.sessionNumber]*len(self.dataForTrial), range(0,len(self.dataForTrial)+1), self.dataForTrial)), columns=['subjectID', 'block', 'session', 'trial', self.variable])
        
        return dataFrame
    
    def AddToFile(self):
        
        dataFrame = pd.DataFrame(list(zip([self.subjectID]*len(self.dataForTrial), [self.blockNumber]*len(self.dataForTrial), [self.sessionNumber]*len(self.dataForTrial), range(0,len(self.dataForTrial)+1), self.dataForTrial)), columns=['subjectID', 'block', 'session', 'trial', self.variable])
        book = load_workbook(self.resultPathName)
        writer = pd.ExcelWriter(self.resultPathName, engine='openpyxl')
        writer.book = book
        writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
        startrow = writer.sheets['Sheet1'].max_row
        dataFrame.to_excel(writer, startrow = startrow, index = False, header = False)
        
        writer.save()
    

In [53]:
metricComputed_count = 'BlinkCount_sections'
metricComputed_freq = 'BlinkFrequency_sections'
metricComputed_durTotal = 'BlinkDurationTotal_sections'
metricComputed_durAvg = 'BlinkDurationAverage_sections'
metricComputed_interDur = 'InterBlinkDuration_sections'

dataFolderName = r'E:\Data\Data' # accessing external hard disk with the data
a = re.compile('(?<=Data\\\\Data\\\\)(.*)(?=\\\\[1-9])')
subjectName_listElement = 3

#dataFolderName = r'C:\DTU\Data\201901_JanuaryExpt' # accessing data saved in the computer
#a = re.compile('(?<=Data\\\\201901_JanuaryExpt\\\\)(.*)(?=\\\\[1-9])')
#subjectName_listElement = 4

resultFileName_count = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Blinks\Subject_Block_Session_Trial_' + metricComputed_count +  '.xlsx'
resultFileName_freq = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Blinks\Subject_Block_Session_Trial_' + metricComputed_freq +  '.xlsx'
resultFileName_durTotal = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Blinks\Subject_Block_Session_Trial_' + metricComputed_durTotal +  '.xlsx'
resultFileName_durAvg = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Blinks\Subject_Block_Session_Trial_' + metricComputed_durAvg +  '.xlsx'
resultFileName_interDur = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Blinks\Subject_Block_Session_Trial_' + metricComputed_interDur +  '.xlsx'



for root, dirs, subfolder in os.walk(dataFolderName):
    
    technique = 'dwell_time'
    
    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'noData' in root or 'Trial' in root or 'trial' in root or 'Nothing' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'Jonas' in root or 'Praktikant' in root or 'Villads' in root:
            continue
            
        if 'Picture' in root:
            continue
        #if '_MS' not in root:
        #    continue
        
        #if 'ac\\4\\' not in root:
        #    continue
        
        
        
        #if '2019-1-16-16-36-17_1stPart_2' not in root:
        #    continue
            
        keysSelected = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            
            if fnmatch.fnmatch(file, 'KeySelection*'):
                try:
                    
                    fKeysSelected = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerKeysSelected = csv.reader(fKeysSelected)
                    keysSelected = list(readerKeysSelected)
                    
                    keysSelected.remove(keysSelected[0])
                except:
                    if fKeysSelected is not None:
                        fKeysSelected.close()
                    else:
                        print('error in opening the KeySelection log file')
                        
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8',  newline='')
                    readerUserKey = csv.reader(fUserKey, quotechar=None)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
                        
            if fnmatch.fnmatch(file, 'phrase*'):
                try:
                    fPhraseLog = open(root + '\\' + file, encoding='utf-8')
                    readerPhraseLog = csv.reader(fPhraseLog, quotechar=None)
                    phraseLog = list(readerPhraseLog)
                    
                except:
                    if fPhraseLog is not None:
                        fPhraseLog.close()
                    else:
                        print('error in opening the phrase log file')
                        
            if fnmatch.fnmatch(file, 'multiKey*'):
                technique = 'multiKey_selection'
            
            if fnmatch.fnmatch(file, 'tobiiGazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerGazeLog = csv.reader(fGazeLog, quotechar=None)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0])
                    gazeLog.remove(gazeLog[-1])
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the scratchpad log file')
            
                    
                     
        if keysSelected is None or userKeys is None or phraseLog is None or gazeLog is None:
            continue
        else:
                
            print('subject path', root)
            subjAndSessionName = '__'.join(root.split('\\')[subjectName_listElement:])
            print(subjAndSessionName)
            subjName = subjAndSessionName.split('__')[0]
            print('subject and session name: ', subjAndSessionName)
            sessionFolderName = root.split('\\')[-1]
            
            # fix userKeys due to comma related file changes
            userKeys_new = FixUserKeys(userKeys)
            
            # need to fix keys selected, where rows get combined because of an inverted comma
            keysSelected_new = FixKeysSelected(keysSelected)
            
            # find start time of typing
            timeTyping = OptiKeyTypingTime(userKeys_new)
            
            # divide complete data into epochs of phrases
            timeStartEndKeys_reading, timeStartEndKeys_writing = FindExptStartEndTimes(keysSelected_new, timeTyping, root)
            
            # use the times of trial start and end, to divide the time of trial into reading and writing
            eventTrialsInKeysSelected_reading, eventTrialsInKeysSelected_readingIndex = \
            FindReadingPartsOfTrial_inKeysSelected(timeStartEndKeys_reading, keysSelected_new, root)
            
            eventTrialsInKeysSelected_writing, eventTrialsInKeysSelected_writingIndex = \
            FindWritingPartsOfTrial_inKeysSelected(timeStartEndKeys_writing, keysSelected_new, eventTrialsInKeysSelected_reading)
            
            
            if sessionFolderName in dict_noGazeData:
                print('no gaze data present')
                blinkCount = [np.nan]*len(eventTrialsInKeysSelected_writing['start'])
                blinkDurationTotal = [np.nan]*len(eventTrialsInKeysSelected_writing['start'])
                blinkDurationAverage = [np.nan]*len(eventTrialsInKeysSelected_writing['start'])
                blinkFrequency = [np.nan]*len(eventTrialsInKeysSelected_writing['start'])
                interBlinkDuration = [np.nan]*len(eventTrialsInKeysSelected_writing['start'])
            
            else:
                # filter the data
                pupilData_filtered, interpolatedIndices, missingData_fromBlinks, missingData_overall, \
                missingDataBlink_startNlength, missingDataAll_startNlength  = FilterPupilSize_wBlinkData(gazeLog, \
                                                                            timeTyping, subjAndSessionName)
                
                """
                print('missing data ratio: ', sum(missingData_overall)/len(missingData_overall))
                print('interpolated data ratio: ', sum(interpolatedIndices)/len(interpolatedIndices))
                print('missing data from blinks ratio: ', sum(missingData_fromBlinks)/len(missingData_fromBlinks))
                
                blinkDurations = [item[1] for item in missingDataBlink_startNlength]
                fig=plt.figure(), plt.hist(blinkDurations)
                plt.title('blinkDurationHistograms')
                pickle.dump(fig, open(r'C:\DTU\Results\201901_Expt\Blinks\blinkDurationHistograms'+'\\'+subjName+'\\'+subjAndSessionName, 'wb'))
                
                missingDataAllDurations = [item[1] for item in missingDataAll_startNlength]
                fig=plt.figure(), plt.hist(missingDataAllDurations)
                plt.title('MissingDataLengthHistograms')
                pickle.dump(fig, open(r'C:\DTU\Results\201901_Expt\Blinks\MissingDataLengthHistograms'+'\\'+subjName+'\\'+subjAndSessionName, 'wb'))
                
                
                """
                #print(missingDataBlink_startNlength)
                
                # divide trials into reading and writing
                eventReading, eventReading_index = EventPartsFromPupilData(eventTrialsInKeysSelected_reading, pupilData_filtered, root)
                eventWriting, eventWriting_index = EventPartsFromPupilData(eventTrialsInKeysSelected_writing, pupilData_filtered, root)
                
                eventTrial, eventTrial_index = CombineReadingWriting(eventReading, eventWriting, eventReading_index, eventWriting_index)
                
                #print('eventReadingIndex: ', eventReading_index)
                #print('')
                #print('eventTrialIndex:  ', eventTrial_index)
                
                # compute blink durations for all trials
                blink = GetBlinkPropertiesForEvents_onSections(eventTrial_index, pupilData_filtered, missingDataBlink_startNlength)
                
                
                blinkCount = blink['blinkCount'].tolist()
                blinkFrequency = blink['blinkFrequency'].tolist()
                blinkDurationAverage = blink['blinkDurationAverage'].tolist()
                blinkDurationTotal = blink['blinkDurationTotal'].tolist()
                interBlinkDuration = blink['interBlinkDuration'].tolist()
                
            if '1stPart' in root:
                print('1stPart')
                blinkCount1 = blinkCount
                blinkFrequency1 = blinkFrequency
                blinkDurationAverage1 = blinkDurationAverage
                blinkDurationTotal1 = blinkDurationTotal
                interBlinkDuration1 = interBlinkDuration
                
                continue
            
            if '2ndPart' in root:
                print('2ndPart')
                blinkCount2 = blinkCount
                blinkFrequency2 = blinkFrequency
                blinkDurationAverage2 = blinkDurationAverage
                blinkDurationTotal2 = blinkDurationTotal
                interBlinkDuration2 = interBlinkDuration
                
                blinkCount = blinkCount1 + blinkCount2
                blinkFrequency = blinkFrequency1 + blinkFrequency2
                blinkDurationAverage = blinkDurationAverage1 + blinkDurationAverage2
                blinkDurationTotal = blinkDurationTotal1 + blinkDurationTotal2
                interBlinkDuration = interBlinkDuration1 + interBlinkDuration2
                
                blinkCount1 = list()
                blinkFrequency1 = list()
                blinkDurationAverage1 = list()
                blinkDurationTotal1 = list()
                interBlinkDuration1 = list()
                
            
            # save the blink count
            dataToSave_count = DataForEveryTrial()
            dataToSave_count.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_count.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_count.sessionNumber = subjAndSessionName[-1]
            dataToSave_count.variable = metricComputed_count
            dataToSave_count.dataForTrial = blinkCount
            dataToSave_count.resultPathName = resultFileName_count
            
            print(dataToSave_count.printInfo())
            
            
            # save the blink frequency
            dataToSave_freq = DataForEveryTrial()
            dataToSave_freq.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_freq.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_freq.sessionNumber = subjAndSessionName[-1]
            dataToSave_freq.variable = metricComputed_freq
            dataToSave_freq.dataForTrial = blinkFrequency
            dataToSave_freq.resultPathName = resultFileName_freq
            
            print(dataToSave_freq.printInfo())
            
            # save the blink duration total
            dataToSave_durTotal = DataForEveryTrial()
            dataToSave_durTotal.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_durTotal.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_durTotal.sessionNumber = subjAndSessionName[-1]
            dataToSave_durTotal.variable = metricComputed_durTotal
            dataToSave_durTotal.dataForTrial = blinkDurationTotal
            dataToSave_durTotal.resultPathName = resultFileName_durTotal
            
            print(dataToSave_durTotal.printInfo())
            
            
            # save the blink duration average
            dataToSave_durAvg = DataForEveryTrial()
            dataToSave_durAvg.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_durAvg.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_durAvg.sessionNumber = subjAndSessionName[-1]
            dataToSave_durAvg.variable = metricComputed_durAvg
            dataToSave_durAvg.dataForTrial = blinkDurationAverage
            dataToSave_durAvg.resultPathName = resultFileName_durAvg
            
            print(dataToSave_durAvg.printInfo())
            
            # save the inter blink duration 
            dataToSave_interDur = DataForEveryTrial()
            dataToSave_interDur.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_interDur.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_interDur.sessionNumber = subjAndSessionName[-1]
            dataToSave_interDur.variable = metricComputed_interDur
            dataToSave_interDur.dataForTrial = interBlinkDuration
            dataToSave_interDur.resultPathName = resultFileName_interDur
            
            print(dataToSave_interDur.printInfo())
            
            
            
            #dataToSave_count.AddToFile()
            #dataToSave_freq.AddToFile()
            #dataToSave_durTotal.AddToFile()
            dataToSave_durAvg.AddToFile()
            dataToSave_interDur.AddToFile()
            
            

subject path E:\Data\Data\ac\1\2019-02-11-11-18-30_1
ac__1__2019-02-11-11-18-30_1
subject and session name:  ac__1__2019-02-11-11-18-30_1
  subjectID block session  trial BlinkCount_sections
0        ac     1       1      0       [23, 7, 8, 7]
1        ac     1       1      1        [9, 7, 5, 5]
2        ac     1       1      2        [5, 2, 3, 2]
3        ac     1       1      3       [19, 6, 6, 5]
4        ac     1       1      4        [5, 2, 2, 2]
  subjectID block session  trial  \
0        ac     1       1      0   
1        ac     1       1      1   
2        ac     1       1      2   
3        ac     1       1      3   
4        ac     1       1      4   

                             BlinkFrequency_sections  
0  [44.83577775502777, 13.639779781208839, 15.601...  
1  [23.102012454808296, 17.965885350274878, 12.83...  
2  [24.013401399052782, 9.762794997088246, 14.644...  
3  [36.8568134459087, 11.65869066491553, 11.62989...  
4  [23.271887792265822, 9.30958128141111, 9.30888...

subject path E:\Data\Data\ac\3_MS\2019-02-14-14-28-49_1
ac__3_MS__2019-02-14-14-28-49_1
subject and session name:  ac__3_MS__2019-02-14-14-28-49_1
  subjectID block session  trial BlinkCount_sections
0        ac  3_MS       1      0       [14, 2, 2, 2]
1        ac  3_MS       1      1        [3, 9, 6, 6]
2        ac  3_MS       1      2       [15, 2, 2, 5]
3        ac  3_MS       1      3      [15, 9, 7, 11]
4        ac  3_MS       1      4       [22, 3, 4, 2]
  subjectID block session  trial  \
0        ac  3_MS       1      0   
1        ac  3_MS       1      1   
2        ac  3_MS       1      2   
3        ac  3_MS       1      3   
4        ac  3_MS       1      4   

                             BlinkFrequency_sections  
0  [36.58837929552473, 5.233025993355889, 5.23879...  
1  [5.847702817663623, 17.809002015187517, 11.852...  
2  [33.69811417115781, 4.497467419877524, 4.49766...  
3  [14.612049833389348, 8.796767546313557, 6.8500...  
4  [50.277087695124614, 6.875230869297698, 

subject path E:\Data\Data\ac\5\2019-02-19-15-07-5_1
ac__5__2019-02-19-15-07-5_1
subject and session name:  ac__5__2019-02-19-15-07-5_1
  subjectID block session  trial BlinkCount_sections
0        ac     5       1      0        [9, 1, 2, 0]
1        ac     5       1      1       [12, 3, 4, 2]
2        ac     5       1      2        [8, 2, 3, 1]
3        ac     5       1      3        [6, 5, 4, 3]
4        ac     5       1      4        [6, 2, 2, 2]
  subjectID block session  trial  \
0        ac     5       1      0   
1        ac     5       1      1   
2        ac     5       1      2   
3        ac     5       1      3   
4        ac     5       1      4   

                             BlinkFrequency_sections  
0  [37.70055385604407, 4.189102803096249, 8.36334...  
1  [40.39499120062442, 10.113423164503594, 13.485...  
2  [28.123956337557786, 7.040491745492795, 10.546...  
3  [14.961285802824467, 12.53754472246682, 10.029...  
4  [22.100604776910664, 7.367177615223732, 7.3672...  


subject path E:\Data\Data\af\2\2019-01-17-15-03-40_1
af__2__2019-01-17-15-03-40_1
subject and session name:  af__2__2019-01-17-15-03-40_1
  subjectID block session  trial BlinkCount_sections
0        af     2       1      0        [4, 3, 3, 4]
1        af     2       1      1        [4, 6, 2, 3]
2        af     2       1      2       [11, 3, 3, 9]
3        af     2       1      3       [11, 6, 6, 5]
4        af     2       1      4        [5, 8, 9, 8]
  subjectID block session  trial  \
0        af     2       1      0   
1        af     2       1      1   
2        af     2       1      2   
3        af     2       1      3   
4        af     2       1      4   

                             BlinkFrequency_sections  
0  [9.577838414363308, 7.268623950955849, 7.24090...  
1  [15.42971763616726, 23.80505091602828, 7.93548...  
2  [29.7224584382082, 8.216117109611993, 8.246379...  
3  [14.607656603208573, 7.922634592907526, 8.1257...  
4  [4.882831573560835, 7.709774903966322, 8.82828...

subject path E:\Data\Data\af\3_MS\2019-02-06-16-19-9_2
af__3_MS__2019-02-06-16-19-9_2
subject and session name:  af__3_MS__2019-02-06-16-19-9_2
  subjectID block session  trial BlinkCount_sections
0        af  3_MS       2      0        [6, 2, 3, 3]
1        af  3_MS       2      1        [3, 2, 2, 1]
2        af  3_MS       2      2       [11, 3, 1, 1]
3        af  3_MS       2      3        [4, 1, 2, 0]
4        af  3_MS       2      4        [5, 0, 4, 2]
  subjectID block session  trial  \
0        af  3_MS       2      0   
1        af  3_MS       2      1   
2        af  3_MS       2      2   
3        af  3_MS       2      3   
4        af  3_MS       2      4   

                             BlinkFrequency_sections  
0  [11.764601692579781, 3.9646778917270256, 5.946...  
1  [11.58133623913349, 8.200217606441216, 8.19980...  
2  [17.81709248212912, 4.862451774608504, 1.61971...  
3  [24.6292401417618, 6.3050894051169175, 12.6136...  
4  [8.392493183407224, 0.0, 6.748379798365159,

...

subject path E:\Data\Data\rh\1\2019-03-07-16-44-5_2
rh__1__2019-03-07-16-44-5_2
subject and session name:  rh__1__2019-03-07-16-44-5_2
  subjectID block session  trial BlinkCount_sections
0        rh     1       2      0    [38, 22, 34, 31]
1        rh     1       2      1    [73, 49, 38, 33]
2        rh     1       2      2    [36, 32, 40, 31]
3        rh     1       2      3    [13, 16, 20, 34]
  subjectID block session  trial  \
0        rh     1       2      0   
1        rh     1       2      1   
2        rh     1       2      2   
3        rh     1       2      3   

                             BlinkFrequency_sections  
0  [30.69722237664331, 18.236158165349647, 28.265...  
1  [50.84232322533391, 34.50540770157883, 26.6384...  
2  [26.73493547962715, 24.68980189404472, 30.9013...  
3  [13.545113380673968, 17.282843857391047, 21.36...  
  subjectID block session  trial  \
0        rh     1       2      0   
1        rh     1       2      1   
2        rh     1       2      2   


subject path E:\Data\Data\rh\3_MS\2019-03-14-13-56-56_2
rh__3_MS__2019-03-14-13-56-56_2
subject and session name:  rh__3_MS__2019-03-14-13-56-56_2
  subjectID block session  trial BlinkCount_sections
0        rh  3_MS       2      0       [18, 5, 9, 7]
1        rh  3_MS       2      1    [53, 39, 39, 31]
2        rh  3_MS       2      2     [39, 16, 10, 9]
3        rh  3_MS       2      3    [18, 22, 15, 17]
  subjectID block session  trial  \
0        rh  3_MS       2      0   
1        rh  3_MS       2      1   
2        rh  3_MS       2      2   
3        rh  3_MS       2      3   

                             BlinkFrequency_sections  
0  [32.95487004475851, 9.218403805725826, 16.5799...  
1  [32.718739253553245, 24.470877151584883, 24.48...  
2  [55.12578124893253, 22.946451539221677, 14.375...  
3  [16.658813372979665, 21.40692437133093, 14.572...  
  subjectID block session  trial  \
0        rh  3_MS       2      0   
1        rh  3_MS       2      1   
2        rh  3_MS       

subject path E:\Data\Data\rh\5\2019-04-15-14-49-27_2
rh__5__2019-04-15-14-49-27_2
subject and session name:  rh__5__2019-04-15-14-49-27_2
  subjectID block session  trial BlinkCount_sections
0        rh     5       2      0    [62, 36, 35, 39]
1        rh     5       2      1    [27, 20, 19, 14]
2        rh     5       2      2    [38, 38, 40, 24]
3        rh     5       2      3    [21, 25, 56, 48]
4        rh     5       2      4    [37, 39, 37, 41]
  subjectID block session  trial  \
0        rh     5       2      0   
1        rh     5       2      1   
2        rh     5       2      2   
3        rh     5       2      3   
4        rh     5       2      4   

                             BlinkFrequency_sections  
0  [44.743230328202785, 26.24418071742842, 25.561...  
1  [33.764878352959244, 25.11046196846854, 23.854...  
2  [22.620443219606972, 22.788280443133704, 23.98...  
3  [8.88243676202019, 10.602512287905313, 24.1089...  
4  [19.905032194775654, 21.246626757511432, 20.11...

subject path E:\Data\Data\ys\2\2019-01-16-15-59-55_2
ys__2__2019-01-16-15-59-55_2
subject and session name:  ys__2__2019-01-16-15-59-55_2
  subjectID block session  trial BlinkCount_sections
0        ys     2       2      0        [3, 0, 0, 0]
1        ys     2       2      1        [7, 1, 2, 2]
2        ys     2       2      2        [6, 1, 2, 1]
3        ys     2       2      3        [7, 0, 0, 1]
4        ys     2       2      4        [6, 1, 0, 2]
  subjectID block session  trial  \
0        ys     2       2      0   
1        ys     2       2      1   
2        ys     2       2      2   
3        ys     2       2      3   
4        ys     2       2      4   

                             BlinkFrequency_sections  
0                [10.215142825573201, 0.0, 0.0, 0.0]  
1  [17.92876874564827, 2.5695775211988003, 5.1381...  
2  [11.60473050076475, 1.9340638313914098, 3.8556...  
3  [27.308647009989244, 0.0, 0.0, 3.9093527942652...  
4  [25.808288719046452, 4.355100999147271, 0.0, 8...

subject path E:\Data\Data\ys\4_MS\2019-01-30-11-22-25_1
ys__4_MS__2019-01-30-11-22-25_1
subject and session name:  ys__4_MS__2019-01-30-11-22-25_1
  subjectID block session  trial BlinkCount_sections
0        ys  4_MS       1      0       [5, 6, 11, 6]
1        ys  4_MS       1      1        [3, 3, 3, 4]
2        ys  4_MS       1      2       [15, 7, 5, 9]
3        ys  4_MS       1      3       [16, 5, 6, 4]
4        ys  4_MS       1      4        [6, 4, 6, 1]
  subjectID block session  trial  \
0        ys  4_MS       1      0   
1        ys  4_MS       1      1   
2        ys  4_MS       1      2   
3        ys  4_MS       1      3   
4        ys  4_MS       1      4   

                             BlinkFrequency_sections  
0  [2.3224373372924596, 2.7873370318560027, 5.110...  
1  [3.4058036635170423, 3.3761417478364604, 3.405...  
2  [12.512269496712173, 5.8371243023542, 4.169971...  
3  [11.972197564206521, 3.7521008950936854, 4.495...  
4  [6.1110199173416655, 4.061156756538881, 